## Pripravili bomo 3D dataset precached podatkov, ki jih bomo uporabili na frontendu.

36 api requestov za salinity IN temperature.

poglejmo, koliko placa zasede ena slika, za en request.

In [5]:
import requests
import time
import json
import sys

# 🎯 API endpoint with parameters (NO /api prefix!)
api_url = "http://localhost:8000/subset"
params = {
    "dataset": "reanalysis",
    "variable": "thetao",
    "min_lon": -73.70759553110591,
    "max_lon": 73.7075955311046,
    "min_lat": 22.17898331112049,
    "max_lat": 77.80754835004322,
    "time": "2011-02-15",
    "depth": 0,
    "stride": 2,
    "fmt": "json"
}

print("=" * 70)
print("🚀 Fetching data from API...")
print("=" * 70)
print(f"📍 URL: {api_url}")
print(f"📊 Parameters:")
for key, value in params.items():
    print(f"   • {key}: {value}")
print()

# ⏱️ Measure request time
start_time = time.time()
try:
    response = requests.get(api_url, params=params)
    response.raise_for_status()
    end_time = time.time()
    
    elapsed_time = end_time - start_time
    
    # 📦 Get data size
    data = response.json()
    data_size_bytes = sys.getsizeof(json.dumps(data))
    data_size_kb = data_size_bytes / 1024
    data_size_mb = data_size_kb / 1024
    
    # 📐 Get shape information from response
    shape = data.get('shape', 'Unknown')
    dims = data.get('dims', 'Unknown')
    
    print("=" * 70)
    print("✅ Request completed successfully!")
    print("=" * 70)
    print()
    print("⏱️  TIME INFORMATION:")
    print(f"   • Total time: {elapsed_time:.3f} seconds")
    print(f"   • Time in milliseconds: {elapsed_time * 1000:.2f} ms")
    print()
    print("📦 DATA SIZE INFORMATION:")
    if data_size_mb >= 1:
        print(f"   • Size: {data_size_mb:.2f} MB ({data_size_bytes:,} bytes)")
    elif data_size_kb >= 1:
        print(f"   • Size: {data_size_kb:.2f} KB ({data_size_bytes:,} bytes)")
    else:
        print(f"   • Size: {data_size_bytes:,} bytes")
    print()
    print("📊 DATA STRUCTURE:")
    print(f"   • Dimensions: {dims}")
    print(f"   • Shape: {shape}")
    if isinstance(shape, list) and len(shape) > 0:
        total_elements = 1
        for dim in shape:
            total_elements *= dim
        print(f"   • Total elements: {total_elements:,}")
    print()
    print("=" * 70)
    print("🎉 Data loaded successfully!")
    print("=" * 70)
    
except requests.exceptions.ConnectionError:
    print("❌ ERROR: Could not connect to API server!")
    print("💡 Make sure the API is running on http://localhost:8000")
    print("   Run: uvicorn app.main:app --reload")
except requests.exceptions.HTTPError as e:
    print(f"❌ HTTP ERROR: {e}")
    print(f"   Status code: {response.status_code}")
    if hasattr(response, 'text'):
        print(f"   Response: {response.text}")
except Exception as e:
    print(f"❌ ERROR: {type(e).__name__}: {e}")

🚀 Fetching data from API...
📍 URL: http://localhost:8000/subset
📊 Parameters:
   • dataset: reanalysis
   • variable: thetao
   • min_lon: -73.70759553110591
   • max_lon: 73.7075955311046
   • min_lat: 22.17898331112049
   • max_lat: 77.80754835004322
   • time: 2011-02-15
   • depth: 0
   • stride: 2
   • fmt: json

✅ Request completed successfully!

⏱️  TIME INFORMATION:
   • Total time: 0.076 seconds
   • Time in milliseconds: 75.68 ms

📦 DATA SIZE INFORMATION:
   • Size: 3.76 MB (3,940,746 bytes)

📊 DATA STRUCTURE:
   • Dimensions: ['latitude', 'longitude']
   • Shape: [334, 885]
   • Total elements: 295,590

🎉 Data loaded successfully!


Zavzame nam minimalno prostora, tako da bomo cachirali v visokem stridu.

Ker lahko pride do problema floating point errorja pri cachanju, odrežemo na 4 decimalke. To naredimo pri /subset api callu, kjer bo v prvem delu funkcije odrezali na 4 decimalke in pogledali v caching sistem ali mamo cachano to.


Cachej salinity in temperature, za 3 stride (2, 4, 8). Stride 1 je prevelik, želi cachati preveč celic, zato ga NE uporabljamo.

We will create a function. This function will read the direct api calls from the cached_data/cache_api_requests.log.

In [1]:
# implement caching system here